In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Alleviating barren plateaus with local cost functions
=====================================================

::: {.meta}
:property=\"og:description\": Local cost functions are cost formulations
for variational quantum circuits that are more robust to barren
plateaus. :property=\"og:image\":
../demonstrations/local\_cost\_functions/Cerezo\_et\_al\_local\_cost\_functions.png
:::

::: {.related}
tutorial\_barren\_plateaus Barren plateaus in quantum neural networks
:::

*Author: Thomas Storwick --- Posted: 09 September 2020. Last updated: 28
January 2021.*

Barren Plateaus
---------------

`Barren plateaus </demos/tutorial_barren_plateaus>`{.interpreted-text
role="doc"} are large regions of the cost function\'s parameter space
where the variance of the gradient is almost 0; or, put another way, the
cost function landscape is flat. This means that a variational circuit
initialized in one of these areas will be untrainable using any
gradient-based algorithm.

In [\"Cost-Function-Dependent Barren Plateaus in Shallow Quantum Neural
Networks\"](https://arxiv.org/abs/2001.00550), Cerezo et al. demonstrate
the idea that the barren plateau phenomenon can, under some
circumstances, be avoided by using cost functions that only have
information from part of the circuit. These *local* cost functions can
be more robust against noise, and may have better-behaved gradients with
no plateaus for shallow circuits.

![Taken from Cerezo et
al..](../demonstrations/local_cost_functions/Cerezo_et_al_local_cost_functions.png){.align-center
width="50.0%"}

Many variational quantum algorithms are constructed to use global cost
functions. Information from the entire measurement is used to analyze
the result of the circuit, and a cost function is calculated from this
to quantify the circuit\'s performance. A local cost function only
considers information from a few qubits, and attempts to analyze the
behavior of the entire circuit from this limited scope.

Cerezo et al. also handily prove that these local cost functions are
bounded by the global ones, i.e., if a global cost function is
formulated in the manner described by Cerezo et al., then the value of
its corresponding local cost function will always be less than or equal
to the value of the global cost function.

In this notebook, we investigate the effect of barren plateaus in
variational quantum algorithms, and how they can be mitigated using
local cost functions.

We first need to import the following modules.


Visualizing the problem
=======================

To start, let\'s look at the task of learning the identity gate across
multiple qubits. This will help us visualize the problem and get a sense
of what is happening in the cost landscape.

First we define a number of wires we want to train on. The work by
Cerezo et al. shows that circuits are trainable under certain regimes,
so how many qubits we train on will effect our results.


Next, we want to define our QNodes and our circuit ansatz. For this
simple example, an ansatz that works well is simply a rotation along X,
and a rotation along Y, repeated across all the qubits.

We will also define our cost functions here. Since we are trying to
learn the identity gate, a natural cost function is 1 minus the
probability of measuring the zero state, denoted here as
$1 - p_{|0\rangle}$.

$$C = \langle  \psi(\theta) | \left(I - |0\rangle \langle 0|\right)  | \psi(\theta)  \rangle   =1-p_{|0\rangle}$$

We will apply this across all qubits for our global cost function, i.e.,

$$C_{G} = \langle  \psi(\theta) | \left(I - |00 \ldots 0\rangle \langle 00 \ldots 0|\right) | \psi(\theta) \rangle  = 1-p_{|00 \ldots 0\rangle}$$

and for the local cost function, we will sum the individual
contributions from each qubit:

$$C_L = \langle \psi(\theta) | \left(I - \frac{1}{n} \sum_j |0\rangle \langle 0|_j\right)|\psi(\theta)\rangle = 1 - \sum_j p_{|0\rangle_j}.$$

It might be clear to some readers now why this function can perform
better. By formatting our local cost function in this way, we have
essentially divided the problem up into multiple single-qubit terms, and
summed all the results up.

To implement this, we will define a separate QNode for the local cost
function and the global cost function.


To analyze each of the circuits, we provide some random initial
parameters for each rotation.


Examining the results:


With this simple example, we can visualize the cost function, and see
the barren plateau effect graphically. Although there are $2n$ (where
$n$ is the number of qubits) parameters, in order to plot the cost
landscape we must constrain ourselves. We will consider the case where
all X rotations have the same value, and all the Y rotations have the
same value.

Firstly, we look at the global cost function. When plotting the cost
function across 6 qubits, much of the cost landscape is flat, and
difficult to train (even with a circuit depth of only 2!). This effect
will worsen as the number of qubits increases.


However, when we change to the local cost function, the cost landscape
becomes much more trainable as the size of the barren plateau decreases.


Those are some nice pictures, but how do they reflect actual
trainability? Let us try training both the local and global cost
functions. To simplify this model, let\'s modify our cost function from

$$C_{L} = 1-\sum p_{|0\rangle},$$

where we sum the marginal probabilities of each qubit, to

$$C_{L} = 1-p_{|0\rangle},$$

where we only consider the probability of a single qubit to be in the 0
state.

While we\'re at it, let us make our ansatz a little more like one we
would encounter while trying to solve a VQE problem, and add
entanglement.


Of course, now that we\'ve changed both our cost function and our
circuit, we will need to scan the cost landscape again.


It seems our changes didn\'t significantly alter the overall cost
landscape. This probably isn\'t a general trend, but it is a nice
surprise. Now, let us get back to training the local and global cost
functions. Because we have a visualization of the total cost landscape,
let\'s pick a point to exaggerate the problem. One of the worst points
in the landscape is $(\pi,0)$ as it is in the middle of the plateau, so
let\'s use that.


After 100 steps, the cost function is still exactly 1. Clearly we are in
an \"untrainable\" area. Now, let us limit ourselves to the local cost
function and see how it performs.


It trained! And much faster than the global case. However, we know our
local cost function is bounded by the global one, but just how much have
we trained it?


Interestingly, the global cost function is still 1. If we trained the
local cost function, why hasn\'t the global cost function changed?

The answer is that we have trained the global cost a *little bit*, but
not enough to see a change with only 10000 shots. To see the effect,
we\'ll need to increase the number of shots to an unreasonable amount.
Instead, making the backend analytic by setting shots to `None`, gives
us the exact representation.


Our circuit has definitely been trained, but not a useful amount. If we
attempt to use this circuit, it would act the same as if we never
trained at all. Furthermore, if we now attempt to train the global cost
function, we are still firmly in the plateau region. In order to fully
train the global circuit, we will need to increase the locality
gradually as we train.


A more thorough analysis
========================

Now the circuit can be trained, even though we started from a place
where the global function has a barren plateau. The significance of this
is that we can now train from every starting location in this example.

But, how often does this problem occur? If we wanted to train this
circuit from a random starting point, how often would we be stuck in a
plateau? To investigate this, let\'s attempt to train the global cost
function using random starting positions and count how many times we run
into a barren plateau.

Let\'s use a number of qubits we are more likely to use in a real
variational circuit: n=10. We will say that after 400 steps, any run
with a cost function of less than 0.9 (chosen arbitrarily) will probably
be trainable given more time. Any run with a greater cost function will
probably be in a plateau.

This may take up to 15 minutes.


In the global case, anywhere between 70-80% of starting positions are
untrainable, a significant number. It is likely that, as the complexity
of our ansatz---and the number of qubits---increases, this factor will
increase.

We can compare that to our local cost function, where every single area
trained, and most even trained in less time. While these examples are
simple, this local-vs-global cost behaviour has been shown to extend to
more complex problems.


References
==========

About the author
================
